<a target="_blank" href="https://colab.research.google.com/github/IngCarlaPezzone/PI1_MLOps_videojuegos/blob/main/JupyterNotebooks/01c_ETL_user_reviews.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## ETL del dataset  `australian_user_reviews`

En esta jupyter notebook se realiza la extracción, transformación y carga del conjunto de datos `australian_user_reviews`.

## Importaciones

In [4]:
import pandas as pd
import ast

%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Extracción, primera exploración y eliminación de duplicados

Se lee el dataset que contiene las reviews que hacen los usuarios y se guarda en un dataframe.

In [6]:
# Ruta al dataset australian_user_reviews
ruta_review = 'data/australian_user_reviews.json'

# Se lee de cada línea del dataset
filas_review = []
with open(ruta_review) as f:
    for line in f.readlines():
        filas_review.append(ast.literal_eval(line))

# Se convierte en dataframe
df_reviews = pd.DataFrame(filas_review)
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Se verifican los tipos de datos para cada columna y la cantidad de nulos.

In [7]:
utils.verificar_tipo_datos(df_reviews)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.0,0.0,0
1,user_url,[<class 'str'>],100.0,0.0,0
2,reviews,[<class 'list'>],100.0,0.0,0


Se verifican si hay o no duplicados por cada 'user_id'.

In [8]:
filas_duplicadas = utils.verifica_duplicados_por_columna(df_reviews, 'user_id')
filas_duplicadas

,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


Se observan 623 filas duplicadas en la columna 'ser_id', pero se revisan si los review dentro de los datos anidados de 'review' la información se encuentra duplicada o si solo se duplica el 'user_id' porque hay mas de un comentario realizado por ese usuario.

In [9]:
# Se revisa un usuario de ejemplo
user_id = '05041129'
user_reviews = filas_duplicadas[filas_duplicadas['user_id'] == user_id]['reviews']

for review_list in user_reviews:
    for review in review_list:
        print(review['review'])
    print('-' * 40)

This game to me it is so good that it is better than any of the games out their and $15 worth it
this is the best third person game ever that i have played
this will be the  number one game if it have more competitive things
----------------------------------------
This game to me it is so good that it is better than any of the games out their and $15 worth it
this is the best third person game ever that i have played
this will be the  number one game if it have more competitive things
----------------------------------------


Se puede ver que los review son exactamente los mismos para cada registro, por lo que se decide borrar los duplicados, dejando la primer ocurrencia de los registros.

In [10]:
df_reviews = df_reviews.drop_duplicates(subset='user_id', keep='first')
utils.verifica_duplicados_por_columna(df_reviews, 'user_id')

'No hay duplicados'

Se revisa la columna 'review' para entender el tipo de dato.

In [11]:
# Se observa el tipo de dato que contiene 'review'
df_reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

Este conjunto contiene 3 columnas y 25799 filas, sin valores nulos. Las columnas son:

* **user_id**: es un identificador único para el usuario.
* **user_url**: es la url del perfil del usuario en streamcommunity.
* **reviews**: contiene una lista de diccionarios. Para cada usuario se tiene uno o mas diccionario con el review. Cada diccionario contiene:
    * **funny**: indica si alguien puso emoticón de gracioso al review.
    * **posted**: es la fecha de posteo del review en formato Posted April 21, 2011.
    * **last_edited**: es la fecha de la última edición.
    * **item_id**: es el identificador único del item, es decir, del juego.
    * **helpful**: es la estadística donde otros usuarios indican si fue útil la información.
    * **recommend**: es un booleano que indica si el usuario recomienda o no el juego.
    * **review**: es una sentencia string con los comentarios sobre el juego.

### Transformación de la columna 'reviews'

La columna 'reviews' se presenta anidada, siendo una lista con uno o mas diccionarios como elementos. Se busca generar una columna por cada diccionario para posteriormente hacer un registro por cada diccionario.

In [12]:
# Se transforma a columnas cada elemento de las listas
df_reviews2 = pd.json_normalize(df_reviews['reviews'])
df_reviews2.head()

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


En esta transformación, se pierde el 'user_id' y 'user_url' al que pertenece cada diccionario, pero aún mantiene la misma posición. Para volver a tener la trazabilidad del usuario, se concatena con el dataframe anterior.

In [13]:
# Se agrega el 'user_id' y 'user_url' a las columnas separadas 
df_reviews2 = pd.concat([df_reviews[['user_id', 'user_url']], df_reviews2], axis=1)
df_reviews2.head()

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


Ahora que se tienen los diccionarios por columnas, con el usuario que genera dicha información, se genera un registro por cada diccionario, manteniendo en cada caso el usuario que lo genera.

In [14]:
# Se utiliza pd.melt para transformar las columnas en filas conservando el 'user_id' y 'user_url'
df_reviews2 = pd.melt(df_reviews2, id_vars=['user_id', 'user_url'], 
                       value_vars=list(range(9)),
                       value_name='reviews')
df_reviews2.head()

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."


Al hacer esto último se puede ver que quedan registros None. Esto ocurre porque hay usuarios que hicieron mas reviews que otros. En este ejemplo se puede ver este caso:

In [15]:
df_reviews2[df_reviews2['user_id']=='76561197970982479']

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
25799,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"{'funny': '', 'posted': 'Posted July 15, 2011...."
51598,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,"{'funny': '', 'posted': 'Posted April 21, 2011..."
77397,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,None
103196,76561197970982479,http://steamcommunity.com/profiles/76561197970...,4,None
128995,76561197970982479,http://steamcommunity.com/profiles/76561197970...,5,None
154794,76561197970982479,http://steamcommunity.com/profiles/76561197970...,6,None
180593,76561197970982479,http://steamcommunity.com/profiles/76561197970...,7,None
206392,76561197970982479,http://steamcommunity.com/profiles/76561197970...,8,None


Se eliminan los registros que tienen None en 'reviews'.

In [16]:
# Se eliminan las filas con valor None
df_reviews2 = df_reviews2.dropna()
# Se verifica que solo queden el 'user_id' con la cantidad de diccionarios que le corresponde
df_reviews2[df_reviews2['user_id']=='76561197970982479']

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
25799,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"{'funny': '', 'posted': 'Posted July 15, 2011...."
51598,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,"{'funny': '', 'posted': 'Posted April 21, 2011..."


En este punto ya es posible convertir cada diccionario en columna.

In [17]:
# Se separan por columnas cada una de las claves de 'reviews'
df_reviews = df_reviews2['reviews'].apply(pd.Series, dtype='object')
df_reviews = df_reviews.add_prefix('reviews_')
df_reviews.head()

,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


En el procesamiento anterior, se puede ver que la columna de 'user_id' y 'user_url' se perdió nuevamente, por lo que se vuelve a concatenar.

In [18]:
# Se une con el 'user_id' y 'user_url'
df_reviews = pd.concat([df_reviews2[['user_id', 'user_url']], df_reviews], axis=1)
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


Se observa que hay valores faltantes en algunas columnas, pero no estan como nulos, probablemente deben tener un espacio. Se compueba esto.

In [19]:
df_reviews['reviews_last_edited'][0]

''

Se reemplazar esos espacios como valores nulos.

In [20]:
df_reviews.replace('', None, inplace=True)
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,None,Posted February 3.,None,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,None,"Posted October 14, 2013.",None,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",None,211420,35 of 43 people (81%) found this review helpful,True,Git gud


Se analizan los tipos de datos y los nulos que quedaron luego de desanidar la columna 'reviews'.

In [21]:
utils.verificar_tipo_datos(df_reviews)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.00,0.00,0
1,user_url,[<class 'str'>],100.00,0.00,0
2,reviews_funny,"[<class 'NoneType'>, <class 'str'>]",13.76,86.24,49498
3,reviews_posted,[<class 'str'>],100.00,0.00,0
4,reviews_last_edited,"[<class 'NoneType'>, <class 'str'>]",10.28,89.72,51499
5,reviews_item_id,[<class 'str'>],100.00,0.00,0
6,reviews_helpful,[<class 'str'>],100.00,0.00,0
7,reviews_recommend,[<class 'bool'>],100.00,0.00,0
8,reviews_review,"[<class 'str'>, <class 'NoneType'>]",99.95,0.05,30


Se observa entre un 86 a 89% de faltantes de datos en las columnas 'reviews_funny' y 'reviews_last_edited' por lo que se decide eliminar estas columnas. Por otra parte hay un 5% de faltantes de datos en la columna propiamente de reviews, pero no se eliminarán esos registros porque se considerarán como un comentario neutral.

In [22]:
# Se eliminan las columnas 'reviews_funny' y 'reviews_last_edited'
df_reviews = df_reviews.drop(columns=['reviews_funny', 'reviews_last_edited'])
df_reviews.columns

Index(['user_id', 'user_url', 'reviews_posted', 'reviews_item_id',
       'reviews_helpful', 'reviews_recommend', 'reviews_review'],
      dtype='object')

## Transformación de la columna 'reviews_posted'

Se necesita que la fecha donde se hizo el posteo de la review este en formato `YYYY-MM-DD`, pero se encuentra como `Posted November 5, 2011.`. Por lo tanto, es necesario procesar la fecha y extraer los elementos relevantes. Se utilizará expresiones regulares para buscar y capturar los valores de año, mes y día dentro de la cadena de texto.

In [23]:
df_reviews['reviews_date'] = df_reviews['reviews_posted'].apply(utils.convertir_fecha)
df_reviews['reviews_date']

0               2011-11-05
1               2014-06-24
2         Formato inválido
3               2013-10-14
4               2014-04-15
                ...       
231291          2014-08-15
231293          2014-08-02
231419          2015-07-31
231499          2015-12-20
231501    Formato inválido
Name: reviews_date, Length: 57397, dtype: object

Se puede observar que hay 9771 registros que contienen un formato inválido distinto a los demas registros. En este caso, no contiene el año del posteo, pero con la función se imputó como 'Formato inválido'. Estos registros no se podrán consultar desde la API, pero las demás columnas serán útiles para aportar información.

In [24]:
df_reviews[df_reviews['reviews_date'] == 'Formato inválido']

,user_id,user_url,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,reviews_date
2,evcentric,http://steamcommunity.com/id/evcentric,Posted February 3.,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,Formato inválido
6,76561198079601835,http://steamcommunity.com/profiles/76561198079...,Posted May 20.,730,0 of 1 people (0%) found this review helpful,True,ZIKA DO BAILE,Formato inválido
7,MeaTCompany,http://steamcommunity.com/id/MeaTCompany,Posted July 24.,730,No ratings yet,True,BEST GAME IN THE BLOODY WORLD,Formato inválido
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,Posted June 16.,252950,0 of 1 people (0%) found this review helpful,True,love it,Formato inválido
10,76561198077246154,http://steamcommunity.com/profiles/76561198077...,Posted June 11.,440,No ratings yet,True,mt bom,Formato inválido
...,...,...,...,...,...,...,...,...
223569,76561198040184950,http://steamcommunity.com/profiles/76561198040...,Posted April 12.,394690,No ratings yet,True,I cannot say much right now due to the game no...,Formato inválido
226105,76561198046474248,http://steamcommunity.com/profiles/76561198046...,Posted March 28.,234140,No ratings yet,True,"Oh what a day .., What a lovely day to play th...",Formato inválido
228109,dmitry_who,http://steamcommunity.com/id/dmitry_who,Posted May 17.,376210,10 of 28 people (36%) found this review helpful,True,░░░░░░░░░░░█▀▀░░█░░░░░░░░░░░▄▀▀▀▀░░░░░█▄▄░░░░░...,Formato inválido
229231,76561198079507136,http://steamcommunity.com/profiles/76561198079...,Posted January 3.,730,No ratings yet,False,got VACed,Formato inválido


Se decide elimina la columna 'reviews_posted' porque no aportan mayor información, ya que solo indica el día y el mes del posteo.

In [25]:
df_reviews = df_reviews.drop('reviews_posted', axis=1)
df_reviews.columns

Index(['user_id', 'user_url', 'reviews_item_id', 'reviews_helpful',
       'reviews_recommend', 'reviews_review', 'reviews_date'],
      dtype='object')

### Columna 'reviews_review'

Esta columna tiene un 5% de valores nulos, por lo que se eliminan.

In [27]:
df_reviews = df_reviews.dropna(subset=['reviews_review'])
# Se verifican nulos
utils.verificar_tipo_datos(df_reviews)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.0,0.0,0
1,user_url,[<class 'str'>],100.0,0.0,0
2,reviews_item_id,[<class 'str'>],100.0,0.0,0
3,reviews_helpful,[<class 'str'>],100.0,0.0,0
4,reviews_recommend,[<class 'bool'>],100.0,0.0,0
5,reviews_review,[<class 'str'>],100.0,0.0,0
6,reviews_date,[<class 'str'>],100.0,0.0,0


### Carga del dataset `australian_user_reviews`

Se guarda el conjunto de datos transformado como `user_review_limpio`. 

In [28]:
archivo_limpio = 'data/user_review_limpio.csv'
df_reviews.to_csv(archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio}')

Se guardó el archivo data/user_review_limpio.csv
